# 03 Rotation Demo
Momentum + volatility-targeted rotation across healthcare ETFs.

- **Momentum:** past winners tend to keep winning in the near term; rank ETFs on 6-month total return.
- **Vol targeting:** scale allocations inversely to volatility to keep portfolio risk steadier.
- **Benchmarks:** buy-and-hold XLV and equal-weighted basket (monthly rebalance).

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure repo root on sys.path
repo_root = Path.cwd().resolve()
if not (repo_root / 'src').exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from src.data.etf_loader import load_clean_prices
from src.signals.rotation_signals import build_monthly_rotation_weights
from src.backtest.engine import run_backtest
from src.analysis.metrics import compute_cagr, compute_annual_vol, compute_sharpe, compute_max_drawdown
from src.analysis.robustness import sweep_momentum_parameters

%matplotlib inline


In [ ]:
tickers = ['XBI', 'XPH', 'IHF', 'IHI', 'XLV']
prices = load_clean_prices()[tickers]
prices = prices.dropna(how='any')
prices.head()


In [ ]:
rotation_weights = build_monthly_rotation_weights(prices, lookback_months=6, top_k=2, target_vol_annual=0.10)
rotation_weights.head()


In [ ]:
rot_result = run_backtest(prices, rotation_weights)
rot_result.equity_curve.tail()


In [ ]:
# Benchmarks
xlv_returns = prices['XLV'].pct_change().fillna(0.0)
xlv_equity = (1 + xlv_returns).cumprod()

month_ends = prices.resample('ME').last().index
ew_monthly = pd.DataFrame(1 / len(tickers), index=month_ends, columns=tickers)
ew_daily_weights = ew_monthly.reindex(prices.index, method='ffill').fillna(0.0)
ew_result = run_backtest(prices, ew_daily_weights)


In [ ]:
def summarize(label, daily_returns, equity_curve):
    cagr = compute_cagr(daily_returns)
    ann_vol = compute_annual_vol(daily_returns)
    sharpe = compute_sharpe(daily_returns)
    max_dd = compute_max_drawdown(equity_curve)
    print(f'{label}: CAGR={cagr:.2%}, Vol={ann_vol:.2%}, Sharpe={sharpe:.2f}, MaxDD={max_dd:.2%}')

summarize('Rotation', rot_result.daily_returns, rot_result.equity_curve)
summarize('Equal-Weight', ew_result.daily_returns, ew_result.equity_curve)
summarize('XLV', xlv_returns, xlv_equity)


In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
rot_result.equity_curve.plot(ax=ax, label='Rotation (mom + vol target)')
ew_result.equity_curve.plot(ax=ax, label='Equal-Weight (monthly)')
xlv_equity.plot(ax=ax, label='XLV Buy & Hold')
ax.set_title('Healthcare Rotation vs Benchmarks')
ax.set_ylabel('Cumulative Wealth')
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
rotation_weights.plot.area(ax=ax, title='Rotation Strategy Weights', stacked=True)
ax.set_ylabel('Weight')
ax.set_ylim(0, 1.05)
plt.tight_layout()
plt.show()


## Robustness: parameter sweep
Sensitivity to lookback/top-k/target vol.

In [ ]:
lookbacks = [3, 6, 9, 12]
top_ks = [1, 2, 3]
target_vols = [0.08, 0.10, 0.12]
sweep_rot = sweep_momentum_parameters(prices, lookbacks, top_ks, target_vols)
sweep_rot.sort_values('sharpe', ascending=False).head()


Interpretation: stable performance across nearby parameters suggests robustness; sharp changes imply sensitivity or overfitting risk.